In [1]:
import pandas as pd
from pandas import json_normalize
import json

data1 = pd.read_json("CPFB_CCPCCPM045T0BAIZV20240109153005725000_20240109_153006_0000001.json")
data1

,totalElements,claims
0,545663,"{'claimID': '113385138', 'hrn': 'xxxxxxxxxxxxx..."
1,545663,"{'claimID': '113792091', 'hrn': 'xxxxxxxxxxxxx..."
2,545663,"{'claimID': '113398390', 'hrn': 'xxxxxxxxxxxxx..."


In [4]:
data1["claims"][0]

{'claimID': '113385138',
 'hrn': 'xxxxxxxxxxxxx',
 'claimType': 'FS',
 'patient': {'name': 'xxxxxxxxxxxxx',
  'uin': 'xxxxxxxxx',
  'typeOfUINCode': 'P',
  'nationality': 'SG',
  'sexCode': 'F',
  'raceCode': 'C',
  'dateOfBirth': '1946-11-24',
  'dateOfAdmission': '2023-11-30T09:00:00.000+08:00',
  'dateOfDischarge': '2023-11-30T17:00:00.000+08:00',
  'referralSourceTypeCode': '01',
  'admittingSourceCode': '9',
  'outcome': '9',
  'principalDoctor': {'name': 'xxxxxxxxxxxxx',
   'registrationNumber': 'M05861J'},
  'finalDiagnosis': {'code': 'E11', 'description': None},
  'otherDiagnosis': [{'code': 'E785', 'description': None}],
  'dischargeWard': 'A',
  'amalgamatedAccount': 'xxxxxxxxx',
  'insurancePolicies': []},
 'items': [{'claimItemID': '1_1810941650734197',
   'typeCode': 'CC',
   'treatmentSettingCode': 'OU',
   'medicalProvider': {'name': 'LIFELINE PUNGGOL MEDICAL CLINIC PTE LTD',
    'code': '0A',
    'typeCode': 'P'},
   'specialtyCode': '07',
   'claimAmount': 130.0,
   'd

In [2]:
def flatten_json(y, prefix=''):
    out = {}
    for key, value in y.items():
        if isinstance(value, dict):
            deeper = flatten_json(value, prefix + key + '.')
            out.update(deeper)
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    deeper = flatten_json(item, prefix + key + f'[{i}].')
                    out.update(deeper)
                else:
                    out[prefix + key + f'[{i}]'] = item
        else:
            out[prefix + key] = value
    return out

In [3]:
df = pd.DataFrame()

for n in range(len(data1)):
    #print(n)
    datapiece = data1['claims'][n]
    json_part = flatten_json(datapiece)
    
    #print(json_part)
    
    df_part = json_normalize(json_part)
    
    df_part["totalElements"] = data1["totalElements"][n]
    
    print(len(df_part))
    
    df = pd.concat([df, df_part])
    
df

1
1
1


,claimID,hrn,claimType,patient.name,patient.uin,patient.typeOfUINCode,patient.nationality,patient.sexCode,patient.raceCode,patient.dateOfBirth,...,claimPayout.payouts[0].payoutDetails[4].amounts.paidAmount,claimPayout.payouts[0].payoutDetails[4].amounts.disburseAmount,claimPayout.payouts[0].payoutDetails[4].claimItemID,claimPayout.payouts[0].payoutDetails[4].payerID,claimPayout.payouts[0].payoutDetails[4].subItemType,claimPayout.utilised[1].id,claimPayout.utilised[1].uin,claimPayout.utilised[1].period,claimPayout.utilised[1].schemeName,claimPayout.utilised[1].amount
0,113385138,xxxxxxxxxxxxx,FS,xxxxxxxxxxxxx,xxxxxxxxx,P,SG,F,C,1946-11-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,113792091,xxxxxxxxxxxxx,FS,xxxxxxxxxxxxx,xxxxxxxxx,P,SG,M,C,1949-11-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,113398390,xxxxxxxxxxxxx,FS,xxxxxxxxxxxxx,xxxxxxxxx,P,SG,M,M,1952-08-21,...,0.0,0.0,4_1873723795605623,None,CHARGE_AMOUNT,PATIENT_MSV_Flexi_2023,xxxxxxxxx,2023,MSV_Flexi,35.25
